# Silver Orders Transformation

## Overview
This module implements Silver-layer transformation for order transactional data.
The Silver Orders table applies schema enforcement, timestamp normalization, and referential readiness, preparing the dataset for analytical joins and aggregations.

## Source Table
- Catalog: `migration_project_db_ws`
- Schema: `bronze`
- Table: `orders`

```sql
migration_project_db_ws.bronze.orders
```

## Target Table
- Catalog: `migration_project_db_ws`
- Schema: `silver`
- Table: `orders`
- Format: Delta Lake
- Processing Type: Batch Transformation

```sql
migration_project_db_ws.silver.orders
```

## Transformation Objectives
The Silver Orders layer ensures:
- Valid and non-null order identifiers
- Proper timestamp parsing
- Numerical consistency for monetary values
- Referential compatibility with Customers Silver table

## Transformation Logic
### Notebook
`04_silver_orders_transformation.py`

---

## Step 1 -- Read from Bronze Layer
```python
df_bronze_orders = spark.read.table(
    "migration_project_db_ws.bronze.orders"
)
```

---

## Step 2 -- Reliable transactional fact table
```python
# Deduplication window
window_spec = Window.partitionBy("order_id").orderBy(col("ingestion_timestamp").desc())

silver_orders_df = (
    bronze_orders_df
    # Data quality filters
    .filter(col("order_id").isNotNull())
    .filter(col("customer_id").isNotNull())
    .filter(col("amount") > 0)

    # Convert order_date from int to date (yyyyMMdd)
    .withColumn(
        "order_date",
        to_date(col("order_date").cast("string"))
    )

    # Deduplication
    .withColumn("row_num", row_number().over(window_spec))
    .filter(col("row_num") == 1)
    .drop("row_num")
)
```

#### Transformations applied:
- Corrected `order_date` from integer to date
- Removed records with null keys
- Filtered invalid values (negative or zero amounts)
- Deduplicated using `order_id`
- Retained latest records using ingestion timestamp

---

### Step 3 -- Write to silver.orders table
```python
silver_orders_df.write.mode("overwrite").saveAsTable("silver.orders")
```


## Summary
- Silver Orders layer enforces clean transactional structure
- Timestamps and numerics are standardized
- Dataset is ready for Gold aggregations
- Designed for scalability and reprocessing